# Project 3: Wrangle OpenStreetMap Data

### Map Area: Atlanta, GA (https://mapzen.com/data/metro-extracts/metro/atlanta_georgia/)

#### I picked Atlanta becuase it is the city I currently live in.

In [1]:
# Note: The schema is stored in a .py file in order to take advantage of the
# int() and float() type coercion functions. Otherwise it could easily stored as
# as JSON or another serialized format.

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


In [1]:
#!jupyter nbconvert --to script schema.ipynb

In [2]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
from collections import defaultdict

import cerberus

import schema

OSM_PATH = "sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [3]:
#create a dictionary of the different tags as the key and how many as the value
def count_tags(filename):
    tags = {}
    for ev, elem in ET.iterparse(filename):
        tag = elem.tag
        if tag not in tags.keys():
            tags[tag] = 1
        else:
            tags[tag] = tags[tag]+1
    return tags

In [11]:
tags = count_tags('sample.osm')
pprint.pprint(tags)

{'member': 3127,
 'nd': 1327804,
 'node': 1171609,
 'osm': 1,
 'relation': 422,
 'tag': 623090,
 'way': 84260}


In [4]:
#provided regular expressions 
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

#checking the 'k' value for each tag for potential problem characters
def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if re.search(lower,k):
            keys["lower"] += 1
        elif re.search(lower_colon,k):
            keys["lower_colon"] += 1
        elif re.search(problemchars,k):
            keys["problemchars"] += 1
            print k
        else:
            keys["other"] += 1
    return keys
    pass

#create a dictionary with 4 tag categories as the key and how many as the value
def process_map_v1(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [16]:
keys = process_map_v1('sample.osm')
pprint.pprint(keys)

towing service
Payment options
Abundance LLC Wealth Advisors
{'lower': 288874, 'lower_colon': 243990, 'other': 90223, 'problemchars': 3}


In [6]:
#number of users that have edited the map data
def process_map_v2(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for e in element:
            if 'uid' in e.attrib:
                users.add(e.attrib['uid'])
    return users
users = process_map_v2('sample.osm')
len(users)

1391

In [5]:
def get_user(element):
    if "uid" in element.attrib:
        return element.attrib["uid"]


def process_map_v3(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        users.update([get_user(element)])
        users.discard(None) 
    return users

In [22]:
#number of users that have edited the map data
users = process_map_v3('sample.osm')
len(users)

1391

In [6]:
osm_file_open = open("sample.osm","r")
osmfile = "sample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(set)

In [7]:
#expected street names for the street type audit
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Way", "Circle", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

In [8]:
#provided street type audit code
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit_1():
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    pprint.pprint(dict(street_types))
    
def audit_2(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types    

In [23]:
audit_1()

{'101': set(['State Rte 101']),
 '138': set(['State Rte 138']),
 '140': set(['S R 140']),
 '20': set(['State Rte 20']),
 '212': set(['State Rte 212']),
 '41': set(['US 41']),
 '5': set(['Old Hwy 5']),
 '78': set(['Highway 78']),
 'Apartment': set(['Woodland Apartment']),
 'Approach': set(['Inverness Approach']),
 'Ave': set(['2nd Ave',
             'Bridge Mill Ave',
             'Confederate Ave',
             'E Foster Ave',
             'E Ponce de Leon Ave',
             'Gail Ave',
             'Highland Ave',
             'Lexington Ave',
             'Lyons Ave',
             'Park Ave',
             'Rolling Hills Ave',
             'Seaboard Ave',
             'Spring Meadows Ave',
             'The General Ave',
             'Transit Ave',
             'W Cooper Ave',
             'W Foster Ave',
             'Washington Ave']),
 'Bend': set(['Glade Bend',
              'Horsehoe Bend',
              'Overlook Bend',
              'Sapphire Bend',
              'Sweetwater Be

In [17]:
st_types = audit_2(osmfile)
pprint.pprint(dict(st_types))

{'101': set(['State Rte 101']),
 '138': set(['State Rte 138']),
 '140': set(['S R 140']),
 '20': set(['State Rte 20']),
 '212': set(['State Rte 212']),
 '30360': set(['The Nalley Way, Atlanta, GA 30360']),
 '330': set(['Highway 330']),
 '41': set(['US 41']),
 '5': set(['Old Hwy 5']),
 '526': set(['526']),
 '53': set(['GA 53']),
 '575': set(['Parkway 575']),
 '710': set(['Dallas Hwy. SW #710']),
 '78': set(['Highway 78']),
 '995': set(['Peachtree Street Northwest #995']),
 'Alley': set(['Hidden Alley']),
 'Apartment': set(['Woodland Apartment']),
 'Approach': set(['Inverness Approach']),
 'Ave': set(['2nd Ave',
             'Bridge Mill Ave',
             'Capital Ave',
             'Confederate Ave',
             'E Foster Ave',
             'E Ponce de Leon Ave',
             'Gail Ave',
             'Georgia Ave',
             'Highland Ave',
             'Lexington Ave',
             'Lyons Ave',
             'North Ave',
             'Park Ave',
             'Polk Ave',
           

## Problems Encountered

#### Street abbreviations:

When looking through the output of audit_1 and audit_2 functions I saw that several street names are abbreviated. For example: Ave, Blvd, Cir, Ct and Dr to name some. There is also some incorrect case abbreviations / spellings, for example: COurt, blvd, circle, dr, drive, lane and place.

These can be updated with a mapping dictionary and using the update_name and test functions.




#### Invalid zip codes:

When looking through the zip code data I noticed 2 records that had invalid zip codes. All zip codes in the Atlanta area start with '30' so it was easy to identify with the below query.

`SELECT id, value
FROM nodes_tags
WHERE key like '%postcode%' AND value not LIKE '30%';`  

`2352501668 | 80083              
3121340792 | Georgia`

id '2352501668' is 4997 Saxony Court, Stone Mountain, GA 80083. The correct zip code is 30083 for this address. This can be updated in the database by an UPDATE statement.

`UPDATE nodes_tags        
SET VALUE = '30083'            
WHERE id = '2352501668' AND key LIKE '%postcode%';`

id '3121340792' is 1420 Cresthaven Lane NW, Lawrenceville, Georgia with the zip code being 'Georgia'. The correct zip code is 30043 for this address. This can also be updated in the database by an UPDATE statement.

`UPDATE nodes_tags        
SET VALUE = '30043'            
WHERE id = '3121340792' AND key LIKE '%postcode%';`

In [9]:
#mapping dictionary for update_name
mapping = { "St": "Street",
            "St.": "Street",
            "street": "Street",
            "Rd": "Road",
            "Rd.": "Road",
            "road": "Road",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "avenue": "Avenue",
            "Dr": "Drive",
            "Dr.": "Drive",
            "drive": "Drive",
            "dr": "Drive",
            "Hwy": "Highway",
            "highway": "Highway",
            "Ct": "Court",
            "Ct.": "Court",
            "court": "Court",
            "COurt": "Court",
            "Cir": "Circle",
            "Cir.": "Circle",
            "circle": "Circle",
            "Hts": "Heights",
            "Ln": "Lane",
            "lane": "Lane",
            "Ter": "Terrace",
            "Trce": "Trace",
            "Trl": "Trail",
            "Vw": "View",
            "Xing": "Crossing",
            "blvd": "Blvd",
            "place": "Place"            
            }

In [10]:
#updating street names per the mapping dictionary
def update_name(name, mapping):

    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            #print 'Before: ' , name
            name = re.sub(m.group(), mapping[m.group()], name)
            #print 'After: ', name
    return name   


def test():
    st_types = audit_2(osmfile)
   
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
            

In [23]:
test()

Waverly Way Northeast => Waverly Way Northeast
Ashland Avenue Northeast => Ashland Avenue Northeast
Mathieson Drive Northeast => Mathieson Drive Northeast
Windemere Drive Northeast => Windemere Drive Northeast
West Ashland Avenue Northeast => West Ashland Avenue Northeast
Parkerton Lane Northeast => Parkerton Lane Northeast
North Fulton Drive Northeast => North Fulton Drive Northeast
Belgrade Avenue Northeast => Belgrade Avenue Northeast
Ivy Gates Northeast => Ivy Gates Northeast
Lindbergh Drive Northeast => Lindbergh Drive Northeast
Old Creek Road Northeast => Old Creek Road Northeast
Bradley Street Northeast => Bradley Street Northeast
Shenandoah Avenue Northeast => Shenandoah Avenue Northeast
Miller Avenue Northeast => Miller Avenue Northeast
Piedmont Road Northeast => Piedmont Road Northeast
Acorn Avenue Northeast => Acorn Avenue Northeast
Baroque Circle Northeast => Baroque Circle Northeast
Parkside Drive Northeast => Parkside Drive Northeast
Rickenbacker Drive Northeast => Ricken

In [11]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [16]:
#shapes the interparse element object and return the dictionary
def shape_element_2(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  #handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for a in node_attr_fields:
            node_attribs[a] = element.attrib[a]

    if element.tag == 'way':
        for b in way_attr_fields:
            way_attribs[b] = element.attrib[b]

    for tag in element.iter('tag'):
        tag_dict = {}
        attributes = tag.attrib
        if problem_chars.search(tag.attrib['k']):
            continue

        if element.tag == 'node':
            tag_dict['id'] = node_attribs['id']
        else:
            tag_dict['id'] = way_attribs['id']
        tag_dict['value'] = attributes['v']

        if tag.attrib['k'] == 'addr:street':
            #update street names
            tag_dict['value'] = update_name(tag.attrib['v'], mapping) 
        else:
            pass
        lower_colon = LOWER_COLON.search(tag.attrib['k'])

        if lower_colon:

            before_colon = re.findall('^(.+?):', tag.attrib['k'])
            after_colon = re.findall('^[a-z|_]+:(.+)', tag.attrib['k'])

            tag_dict['type'] = before_colon
            tag_dict['key'] = after_colon

        else:
            tag_dict['key'] = attributes['k']
            tag_dict['type'] = 'regular'

        tags.append(tag_dict)

    if element.tag == 'way':
        count = 0 
        for nd in element.iter('nd'):
            way_node_dict = {}
            way_node_dict['id'] = element.attrib['id']
            way_node_dict['node_id'] = nd.attrib['ref']
            way_node_dict['position'] = count
            count += 1
            way_nodes.append(way_node_dict)



    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [17]:
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map_2(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element_2(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [19]:
process_map_2(OSM_PATH, validate=False)

# Data Overview

### Statistics from the Atlanta OpenStreetMap dataset

`sample.osm:          248 MB                                
project3.db:         146 MB                                      
nodes.csv:           97 MB                            
nodes_tags.csv:      8 MB            
ways.csv:            5 MB             
ways_nodes.csv:      31 MB                  
ways_tags.csv:       17 MB`



#### Number of unique users:

``` sql
SELECT COUNT(DISTINCT(e.uid))                   
FROM (SELECT uid FROM nodes 
      UNION ALL 
      SELECT uid FROM ways) e;
```

1381

#### Number of nodes:

```sql
SELECT COUNT(*) FROM nodes;
```

1171609

#### Number of ways:

``` sql
SELECT COUNT(*) FROM ways;
```

84260

#### Top 5 contributing users:

```sql
SELECT u.user, COUNT(*) as num         
FROM (SELECT user FROM nodes 
      UNION ALL 
      SELECT user FROM ways) u           
GROUP BY u.user         
ORDER BY num DESC          
LIMIT 5;
```


`Liber                         | 532818                   
Saikrishna_FultonCountyImport | 241015                    
woodpeck_fixbot               | 148812                   
Jack the Ripper               | 35026                
afonit                        | 33632                       `

#### Top 5 types of 'natural' key:

```sql
SELECT tags.value, COUNT(*) as count               
FROM (SELECT * FROM nodes_tags               
      UNION ALL              
      SELECT * FROM ways_tags) tags            
WHERE tags.key = 'natural'                                 
GROUP BY tags.value              
ORDER BY count DESC            
LIMIT 5;      
```

`water   | 3180                
wood    | 810        
tree    | 561            
wetland | 137        
peak    | 25`


#### Most popular religions:

```sql
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='religion'
GROUP BY value
ORDER BY num DESC;
```      

`christian | 382                     
muslim    | 1`


#### Top 5 cusines:

```sql
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='cuisine'
GROUP BY value
ORDER BY num DESC
LIMIT 5;
```


`
burger   | 16      
pizza    | 11               
mexican  | 9            
chinese  | 7          
american | 6    `          



### Improvements

#### 'place' key in the dataset:

```sql
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='place'
GROUP BY value
ORDER BY num DESC;
```

`hamlet        | 193            
neighbourhood | 140           
village       | 13            
locality      | 6          
county        | 5          
suburb        | 5         
island        | 4    `

I thought it was odd to see the top place was a hamlet, the 3rd most common place was a village and neighbourhood is the British spelling. So I investigated where OpenStreetMaps was founded and it was as I guessed, in the UK. I would say that in the US we don't use the words hamlet or village to describe a place, at least not as widely as in the UK. 

I would recommend using more common words to the region to describe places. What does a hamlet mean vs. a village in the US? This would allow for better understanding of place types in the US, although I do understand using these place constructs for continuity throughout the entire map dataset. If this was implemented it would require common words to the region to be mapped back to the original meaning of hamlet/village.

Benefits:

* Better understanding of the place type keys in different areas of the world
* Add meaning to the place keys in different areas of the world

Anticipated Problems:

* Going against the standard place keys can create continuity issues and create confusion
* Common words to a region would have to be mapped back to the original place keys they are referring to

I also noticed that there are apparently 4 islands in the dataset. I am not aware of any island in the Atlanta area so I investigated the 4 nodes.

``` sql
SELECT nt.id, n.lat, n.lon
FROM nodes_tags nt
JOIN nodes n
ON nt.id = n.id
WHERE nt.key = 'place' AND nt.value = 'island';
```

`358686776  | 32.9934577 | -85.1860533  -> Hairston Island (can't visibly see the island from Google Maps)                     
358697497  | 32.8515216 | -84.4668655  -> Owens Island                
358705587  | 33.1481772 | -85.0546617  -> Swanson Island (can't visibly see the island from Google Maps)           
3473397106 | 33.5887561 | -84.20201    -> An unnamed island, verified with Google Maps earth shot, maps shows as a body of water`

It is strange that both OpenStreetMaps and Google Maps incorrectly identify 2 islands (and both show the unnamed island as a body of water), I would assume one is based on the other's incorrect assumption and neither have been audited.




## References:

SQL sample project: https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md

SQL schema: https://gist.github.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f

David Venturi's submission: https://gist.github.com/davidventuri/5c902f01ca2e483f5208111413adb13a

Ly Vinh Hung's submission: https://github.com/lyvinhhung/Udacity-Data-Analyst-Nanodegree/blob/master/p3%20-%20Wrangle%20OpenStreetMap%20Data/P3%20-%20Data%20Wrangling%20with%20MongoDB.ipynb

SQLite documentation: http://www.sqlite.org/cli.html

Various posts in the Udacity forums: https://discussions.udacity.com/c/nd002-p3-data-wrangling